In [50]:
import requests
import time
import pickle
from bs4 import BeautifulSoup
import re
import json
import pandas as pd
import os
from tqdm import tqdm

In [51]:
url = 'https://everynoise.com/everynoise1d.cgi?scope=all'
response = requests.get(BASE_URL)
soup = BeautifulSoup(response.content, 'html.parser')

In [52]:
everynoise_dict = dict()
count = 0

all_links = soup.find_all('a')
for link in all_links:
    href = link.get('href')
    if '?root=' in href:
        genre_id = re.findall(r'(?<=\?root=).+(?=&scope=all)', href)[0]
        
        # remove special characters from genre_id to match id in wikidata
        genre_id = re.sub(r'%20', '', genre_id)  # remove space
        genre_id = re.sub(r'%26', '', genre_id)  # remove &
        genre_id = re.sub(r'%27', '', genre_id)  # remove '
        genre_id = re.sub(r'%2B', '', genre_id)  # remove +
        genre_id = re.sub(r'%C3%BA', '', genre_id)  # remove ú
        genre_id = re.sub(r'%3A', '', genre_id)  # remove :
        genre_id = re.sub(r'-', '', genre_id)
        
        # check that genre_id is only made of alpha-num characters
        non_alphanum = re.findall(r'\W', genre_id)
        if non_alphanum:
            print(genre_id)
        
        genre_name = link.text
        everynoise_dict[count] = {'everynoise_id': genre_id, 'everynoise_name': genre_name}
        count += 1
        
len(everynoise_dict)

5602

In [53]:
everynoise_df = pd.DataFrame.from_dict(everynoise_dict, orient='index')
everynoise_df = everynoise_df.sort_values(by=['everynoise_id']).reset_index(drop=True)
everynoise_df.to_csv('data/data_everynoise_genres.csv', encoding='utf-8')
everynoise_df

,everynoise_id,everynoise_name
0,21stcenturyclassical,21st century classical
1,432hz,432hz
2,48g,48g
3,5thwaveemo,5th wave emo
4,8bit,8-bit
...,...,...
5597,zouglou,zouglou
5598,zouk,zouk
5599,zoukriddim,zouk riddim
5600,zurichindie,zurich indie
